In [4]:
from langchain_community.chat_models import ChatZhipuAI
from dotenv import load_dotenv
import os


load_dotenv()
key = os.getenv('ZHIPUAI_API_KEY')

In [6]:
llm = ChatZhipuAI(
    temperature=0.1,
    api_key=key,
    model_name="glm-4",
)

from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///G:/Python Project/llm+sql/ali_langchain.db")

def get_schema(_):
    return db.get_table_info()
    
def run_query(query):
    return db.run(query)

result = run_query("SELECT COUNT(*) FROM alidata")
print(result)
print(db.get_table_info())

[(182880,)]

CREATE TABLE alidata (
	"index" INTEGER, 
	user INTEGER, 
	brand INTEGER, 
	behavr INTEGER, 
	date TEXT
)

/*
3 rows from alidata table:
index	user	brand	behavr	date
0	10944750	13451	0	06/04
1	10944750	13451	2	06/04
2	10944750	13451	2	06/04
*/


In [ ]:
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate

template_sql = (
    "请通过写sql代码来回答对应问题，并且需要基于如下数据库信息：{info}\n"
    "需要回答的问题是：{question}\n"
    "注意仅需要通过sql代码回答，不需要文字\n"
    "代码形式如下：```sql\n...\n```"
)

t_sql=PromptTemplate.from_template(template_sql)

def get_sql(x):
    return x.split("```sql")[1].split("```")[0]

chain_sql=({"info":get_schema,"question":RunnablePassthrough()}
           |t_sql
           |llm
           |StrOutputParser()
           |RunnableLambda(get_sql))
template_sql0 = (
    "请通过综合如下的数据库信息回答问题。问题,sql代码，sql代码的执行结果给出问题的自然语言回答。\n"
    "数据库信息{info}\n"
    "需要回答的问题是：{question}\n"
    "sql代码: {query}\n"
    "sql代码执行结果: {res}"
)

t_sql0=PromptTemplate.from_template(template_sql0)
chain_sql0=({"info":get_schema,"question":RunnablePassthrough(),"query":chain_sql}
            |RunnablePassthrough.assign(res=lambda x: run_query(x["query"]))
            |t_sql0
            |llm
            |StrOutputParser())

In [ ]:
chain_sql.invoke("帮我查询具有最多的behavior是哪个用户？有几条?")

'\nSELECT user, behavr, COUNT(*) as behavr_count\nFROM alidata\nGROUP BY user, behavr\nORDER BY behavr_count DESC, behavr DESC\nLIMIT 1;\n'

In [ ]:
chain_sql0.invoke("帮我查询具有最多的behavior是哪个用户？有几条?")

'根据提供的SQL代码执行结果，查询到具有最多behavior记录的用户是用户编号为2940000，其行为（behavr）为0的记录共有2376条。这是在所有用户中，针对单一行为类型数量最多的记录。'